Test R-CNN Code (Tensorflow)

1. Create training data
Generate Region Proposals
Region proposals are generated using the selective search algorithm. Selective search is an algorithm that groups similar elements based texture, color, shape or size. It is hierachical, so it starts to define very fine-grained elements and combines them to bigger elements. Ultimately it will yield multiple region proposals. It is noteworthy that this algorithm has nothing to do with machine learning, later methods replace that element with neural networks.

Check region proposals
Since we have the real object detection rectangles for an image, we can compare the region proposal to the ground truths. We use a method called intersection over union (IoU). It measures the overlap between the predicted proposal and ground truth. If it is bigger than 50%, we classify the prediction as a positiv sample. In this way we build our positive and negative training classes.

Extract images
In this implementation we crop the images (using the region proposals) and save them as images in a folder.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

import pathlib
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import random
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import requests
import cv2
from PIL import Image

In [ ]:
def extract_ground_truth(path):
    """Extracts ground truths like boxes, labels and ids from xml files."""

    boxes = []
    labels = []
    ids = []

    tree = ET.parse(path)
    id_i = path.split("/")[-1].split(".")[1]
    objects = tree.getroot().findall("object")

    for object_i in objects:
        object_name = object_i.find("name").text
        if object_name=="cat":
            instances = object_i.findall("bndbox")
            for instance in instances:
                xmin = int(float(instance.find("xmin").text))
                xmax = int(float(instance.find("xmax").text))
                ymin = int(float(instance.find("ymin").text))
                ymax = int(float(instance.find("ymax").text))
                box = np.array([xmin, xmax, ymin, ymax])
                boxes.append(box)
                labels.append(object_name)
                ids.append(id_i)
        return np.array(boxes), np.array(labels), np.array(ids) #We will only consider last item, can be changed in future iterations

In [ ]:
def region_proposals(image):
    """Generates region proposals using Selective Search"""
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()

    prediction_rects = _restructure_rectangles_xx(rects)
    return prediction_rects

def _restructure_rectangles_xx(old_rects):
    """Converts rectangles from (x, y, w, h) to (x1, x2, y1, y2)"""
    x1 = old_rects[:,0]
    x2 = old_rects[:,2]+old_rects[:,0]
    y1 = old_rects[:,1]
    y2 = old_rects[:,1]+old_rects[:,3]

    rects_new = old_rects.copy()
    rects_new[:,0] = x1
    rects_new[:,1] = x2
    rects_new[:,2] = y1
    rects_new[:,3] = y2
    return rects_new

In [ ]:
def images_to_file(image, boxes, labels, predicted_rectangles, image_id, path="."):
    """Crops boxes and writes images to files."""

    if not os.path.exists(f"{path}/{labels[0]}"):
        os.makedirs(f"{path}/{labels[0]}")
    if not os.path.exists(f"{path}/no_category"):
        os.makedirs(f"{path}/no_category")

    # Iterate through boxes
    for box, label, i_id in zip(boxes, labels, image_id):
        i_pos = i_neg = 0
        for rect in predicted_rectangles:
            iou = _get_iou(box,rect)
            if iou > 0.5:
                i_pos +=1
                file_name = f"{path}/{label}/{i_id}_{i_pos}.jpg"
                cropped_image = image[rect[2]:rect[3], rect[0]:rect[1]]
                img = Image.fromarray(cropped_image, 'RGB')
                img2 = img.resize((227,227), Image.ANTIALIAS)
                img2.save(file_name)
            elif iou < 0.1 and i_neg < 50:
                if (rect[3]-rect[2])>20 and (rect[1]-rect[0])>20:
                    i_neg += 1
                    file_name = f"{path}/no_category/{label}_{i_id}_{i_neg}.jpg"
                    cropped_image = image[rect[2]:rect[3], rect[0]:rect[1]]
                    img = Image.fromarray(cropped_image, 'RGB')
                    img2 = img.resize((227,227), Image.ANTIALIAS)
                    img2.save(file_name)


In [ ]:
# Inspired by https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
def _get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : array
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : array
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1[0] < bb1[1]
    assert bb1[2] < bb1[3]
    assert bb2[0] < bb2[1]
    assert bb2[2] < bb2[3]

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[2], bb2[2])
    x_right = min(bb1[1], bb2[1])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[1] - bb1[0]) * (bb1[3] - bb1[2])
    bb2_area = (bb2[1] - bb2[0]) * (bb2[3] - bb2[2])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
metadata_paths = sorted(glob("../input/asirra-cats-vs-dogs-object-detection-dataset/Asirra: cat vs dogs/cat*.xml"))
images_paths = sorted(glob("../input/asirra-cats-vs-dogs-object-detection-dataset/Asirra: cat vs dogs/cat*.jpg"))

In [ ]:
for metadata_path, image_path in tqdm(zip(metadata_paths, images_paths)):
    if os.stat(image_path).st_size < 100000:
        image = cv2.imread(image_path)

        truth_recangles, truth_label, image_id = extract_ground_truth(metadata_path)

        if truth_label:
            predicted_rectangles = region_proposals(image)
            images_to_file(image, truth_recangles, truth_label, predicted_rectangles, image_id, "./training")

2. Train Neural Network
In the original paper the authors use a neural network to output a 4096-dimensional representation of an image. This implementations classifies the output (region proposals of cat vs. no cat) directly. It uses transfer learning by re-training a MobileNetV2. Futhermore various steps to augment the image data by shifting, rotating or flipping pixels are used.

I simplified the training (no verification data, no hyperparameter search) to save some of my time ;)

In [ ]:
data_train = keras.utils.image_dataset_from_directory("./training", image_size=(227,227))

In [ ]:
data_train.class_names

In [ ]:
# Inspired by https://pyimagesearch.com/2020/07/13/r-cnn-object-detection-with-keras-tensorflow-and-deep-learning/
# Create neural network
class_weight = {0: 5, 1: 1}

input_l = keras.layers.Input(shape=(227, 227, 3))
input_layer = keras.layers.Rescaling(1./255)(input_l)
input_layer = keras.layers.ZeroPadding2D((16,16))(input_layer)
input_layer = keras.layers.RandomCrop(227,227)(input_layer)
input_layer = keras.layers.RandomFlip("horizontal")(input_layer)

headModel = keras.applications.MobileNetV2(weights="imagenet", include_top=False,input_tensor=input_layer)

for layer in headModel.layers:
    layer.trainable = False

headModel = headModel.output

baseModel = keras.layers.AveragePooling2D(pool_size=(7, 7))(headModel)
baseModel = keras.layers.Flatten(name="flatten")(baseModel)
baseModel = keras.layers.Dense(128, activation="relu")(baseModel)
baseModel = keras.layers.Dropout(0.3)(baseModel)
output = keras.layers.Dense(2, activation="softmax")(baseModel)


model = keras.models.Model(inputs=input_l, outputs=output)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

history = model.fit(data_train, epochs=10, batch_size=64, class_weight=class_weight)

In [ ]:
model.summary()

3. Prediction
Download prediction data
For demonstration purpose I use an example image from Google Images search to verify the algorithm.

In [ ]:
prediction_sample = "https://i.ytimg.com/vi/K1Xkt1E7PJY/maxresdefault.jpg"

response = requests.get(prediction_sample)

with open("./prediction.jpg", 'wb') as f:
    f.write(response.content)

prediction = plt.imread("./prediction.jpg")

plt.imshow(prediction)
plt.show()